In [65]:
import pandas as pd
import numpy as np
from scipy.optimize import bisect
from scipy.stats import norm

# file_path = './SPX_hedging.csv'
# data = pd.read_csv(file_path, usecols=['S', 'C_BS', 'D_BS', 'C_mkt', 'D_Blm', 'TTM', 'Moneyness', 'D_Optimal', 'R', 'Dividend', 'K'])
# data.to_csv('./SPX_options.csv', index=False)
data = pd.read_csv('./SPX_options_with_IV.csv')
data['TTM'] = data['TTM']/252  # Assuming 252 trading days
# print(data.columns)
# Ignoring SPX dividend might introduce some error

In [66]:
data.describe()

,S,Dividend,C_BS,D_BS,C_mkt,D_Blm,R,TTM,Moneyness,D_Optimal,K,TTM_years,R_decimal,Implied_Volatility
count,85.000000,85.000000,85.000000,85.000000,85.000000,85.000000,85.00000,85.000000,85.000000,85.000000,85.0,85.000000,85.000000,85.000000
mean,5718.486941,1.345927,300.508027,0.591168,315.221176,0.609341,4.80564,0.670121,18.458824,0.060418,5700.0,0.670121,0.048056,0.102993
std,201.911555,0.047218,93.940648,0.120690,95.263399,0.120187,0.10632,0.139251,201.966260,5.258367,0.0,0.139251,0.001063,0.008357
min,5186.330000,1.276500,106.223538,0.291429,111.850000,0.312000,4.59072,0.428571,-514.000000,-39.819644,5700.0,0.428571,0.045907,0.085510
25%,5597.120000,1.311000,237.004250,0.526772,257.450000,0.549000,4.76095,0.555556,-103.000000,0.424070,5700.0,0.555556,0.047609,0.097324
50%,5728.800000,1.336400,295.415563,0.591379,311.800000,0.610000,4.77997,0.670635,29.000000,0.580349,5700.0,0.670635,0.047800,0.102782
75%,5853.980000,1.372100,368.601090,0.669328,383.750000,0.686000,4.88116,0.785714,154.000000,0.765608,5700.0,0.785714,0.048812,0.108534
max,6049.880000,1.481600,466.011822,0.800804,487.300000,0.818000,5.01499,0.904762,350.000000,8.794966,5700.0,0.904762,0.050150,0.127875


In [79]:
# Black-Scholes call option pricing formula
def black_scholes_call_price(S, K, T, r, sigma):
    if T <= 0:
        return max(S - K, 0)  # Payoff at maturity
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)

# Function to calculate implied volatility
def implied_volatility(C_mkt, S, K, T, r):
    def objective(sigma):
        price = black_scholes_call_price(S, K, T, r, sigma)
        print("price:", price)
        output = price - C_mkt
        print(output)
        return output
    try:
        return bisect(objective, 1e-6, 10)  # Expanded range
    except ValueError:
        return np.nan

# Function to calculate call Delta using the Black-Scholes model
def black_scholes_call_delta(S, K, T, r, sigma):
    d1 = (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    return norm.cdf(d1)

# Crank-Nicholson method for solving the Black-Scholes PDE
def crank_nicholson(S, K, r, T, sigma, N=100, M=100):
    dt = T / N  # Time step size
    S_max = S * np.exp((r - (sigma**2)/2) * T + 3 * sigma * np.sqrt(T))
    S_min = S * np.exp((r - (sigma**2)/2) * T - 3 * sigma * np.sqrt(T))
    dS = (S_max - S_min)/M
    print(f"S: {S}, S_min: {S_min}, S_max: {S_max}")
    # dS = (S_max - S_min)/M + S_min
    
    # dx = sigma * np.sqrt(3 * dt)  # Price step size
    # pu = 0.5 * dt * ((sigma / dx)**2 + (r - 0.5 * sigma**2) / dx)
    # pm = 1 - dt * (sigma / dx)**2 - r * dt
    # pd = 0.5 * dt * ((sigma / dx)**2 - (r - 0.5 * sigma**2) / dx)

    # Initialize the price grid
    S_grid = np.linspace(S_min, S_max, M + 1)
    # print(S_grid)
    V = np.maximum(S_grid - K, 0)  # Option value at maturity
    V[0] = S_min
    V[-1] = S_max
    # Coefficinents for Crank-Nicolson
    alpha = 0.25 * dt * ((sigma**2 * S_grid**2) / (dS**2) - r * S_grid / dS)
    beta = -0.5 * dt * (sigma**2 * S_grid**2 / (dS**2) + r)
    gamma = 0.25 * dt * ((sigma**2 * S_grid**2) / (dS**2) + r * S_grid / dS)
    
    # Implicit matrix
    A = np.zeros((M-1, M-1))
    B = np.zeros((M-1, M-1))
    for i in range(1, M):
        if i > 1:
            A[i-1, i-2] = -alpha[i]
            B[i-1, i-2] = alpha[i]
        A[i-1, i-1] = 1 - beta[i]
        B[i-1, i-1] = 1 + beta[i]
        if i < M-1:
            A[i-1, i] = -gamma[i]
            B[i-1, i] = gamma[i]

    A_matrix = pd.DataFrame(A)
    B_matrix = pd.DataFrame(B)

    print(f" ----------------- \n A: {A_matrix} \n ------------------- \n B: {B_matrix}")
    
    # Time stepping
    for _ in range(N):
        V_inner = V[1:M]
        V_inner = np.linalg.solve(A, B @ V_inner)
        V[1:M] = V_inner
        V[0] = 0  # Boundary condition at S = 0
        V[-1] = S_max - K * np.exp(-r * (T - _ * dt))  # Boundary condition at S -> infinity
        
    print(f"S: {S}, S_grid: {S_grid}, V: {V}")

    return V


# C_mkt = 10.5
# S = 100
# K = 110
# T = 0.5
# r = 0.02

# # Calculate implied volatility
# sigma_imp = implied_volatility(C_mkt, S, K, T, r)
# print(f"Implied Volatility: {sigma_imp:.4f}")


In [80]:
# Initialize strategy variables
position = 0  # Current asset position
cash = 100000  # Initial cash balance
portfolio_values = []  # Store portfolio values over time
predicted_prices = []  # Store predicted option prices

# Parameters for Crank-Nicholson
M = 1000  # Number of grid points
epsilon = 1e-4  # Small perturbation for Delta calculation

# Iterate over each row of the data to implement the hedging strategy
for i in range(len(data)):
    S = data['S'][i]  # Current stock price
    K = data['K'][i]  # Strike price
    T = data['TTM'][i] / 252  # Convert time to expiration to years
    r = data['R'][i] - data['Dividend'][i]  # Risk-free rate minus dividend
    C_mkt = data['C_mkt'][i]  # Market option price
    sigma = data['Implied_Volatility'][i]

    # Precompute the Crank-Nicholson grid once
    V_grid = crank_nicholson(S, K, r, T, sigma, M)
    print(f"V_grid: {V_grid}, Type: {type(V_grid)}")


    # Calculate Delta using central differences
    delta_index = M // 2  # Assuming S corresponds to the middle of the grid
    delta = (V_grid[delta_index + 1] - V_grid[delta_index - 1]) / (2 * epsilon)

    # Predicted price corresponds to the center of the grid
    predicted_price = V_grid[delta_index]
    predicted_prices.append(predicted_price)

    # Determine the target position based on Delta
    target_position = -delta
    position_change = target_position - position

    # Update cash and asset position
    cash -= position_change * S
    position = target_position

    # Record portfolio value
    portfolio_values.append(position * S + cash)

# Output the portfolio values and predicted prices
results = {
    "Portfolio Values": portfolio_values,
    "Predicted Prices": predicted_prices,
}


S: 5186.33, S_min: 5131.99119128789, S_max: 5373.432019212113
 ----------------- 
 A:           0         1         2         3         4         5    6    7    8   \
0   1.132756 -0.073012  0.000000  0.000000  0.000000  0.000000  0.0  0.0  0.0   
1  -0.059798  1.132881 -0.073077  0.000000  0.000000  0.000000  0.0  0.0  0.0   
2   0.000000 -0.059857  1.133006 -0.073143  0.000000  0.000000  0.0  0.0  0.0   
3   0.000000  0.000000 -0.059916  1.133131 -0.073208  0.000000  0.0  0.0  0.0   
4   0.000000  0.000000  0.000000 -0.059976  1.133256 -0.073274  0.0  0.0  0.0   
..       ...       ...       ...       ...       ...       ...  ...  ...  ...   
94  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.0  0.0  0.0   
95  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.0  0.0  0.0   
96  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.0  0.0  0.0   
97  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.0  0.0  0.0   
98  0.000000  0.000000 

IndexError: index 501 is out of bounds for axis 0 with size 101

In [45]:
print(results)

NameError: name 'results' is not defined

In [44]:
# # Initialize strategy variables
# position = 0  # Current asset position
# cash = 100000  # Initial cash balance
# portfolio_values = []  # Store portfolio values over time
# predicted_prices = []  # Store predicted option prices

# # Store the previous implied volatility
# sigma_prev = None
# M = 1000

# # Iterate over each row of the data to implement the hedging strategy
# for i in range(len(data)):
#     S = data['S'][i]  # Current stock price
#     K = data['K'][i]  # Strike price
#     T = data['TTM'][i] / 252  # Convert time to expiration to years
#     r = data['R'][i] - data['Dividend'][i]  # Risk-free rate minus dividend
#     C_mkt = data['C_mkt'][i]  # Market option price
#     sigma = data['Implied_Volatility'][i]

#     # Calculate implied volatility
#     # sigma = implied_volatility(C_mkt, S, K, T, r)
#     if np.isnan(sigma):
#         if sigma_prev is not None:
#             sigma = sigma_prev
#         else:
#             print("skipping")
#             continue
#     sigma_prev = sigma

#     # Calculate Delta and predicted price using Crank-Nicholson
#     epsilon = 1e-4  # Small perturbation
#     V_grid = crank_nicholson(S, K, r, T, sigma, 1000)
#     delta = (V_grid[M+1] - V_grid[M-1]) / (2 * epsilon)
#     predicted_price = V_grid[M]
#     predicted_prices.append(predicted_price)

#     # Determine the target position based on Delta
#     target_position = -delta
#     position_change = target_position - position

#     # Update cash and asset position
#     cash -= position_change * S
#     position = target_position

#     # Record portfolio value
#     portfolio_values.append(position * S + cash)


IndexError: invalid index to scalar variable.

In [22]:
# Convert portfolio values to a DataFrame for further analysis
portfolio_values = pd.DataFrame(portfolio_values, columns=['Portfolio Value'])
# print(portfolio_values.head())

# Print predicted prices
predicted_prices_df = pd.DataFrame(predicted_prices, columns=['Predicted Price'])
print(predicted_prices_df.head())
print("---------------")
print(data.head()['C_mkt'])

# Calculate risk metrics
portfolio_values['Returns'] = portfolio_values['Portfolio Value'].pct_change().dropna()
volatility = portfolio_values['Returns'].std()  # Calculate return volatility
cumulative_returns = (1 + portfolio_values['Returns']).cumprod()
drawdown = cumulative_returns.cummax() - cumulative_returns  # Calculate drawdown
max_drawdown = drawdown.max()
# Output risk metrics
volatility, max_drawdown

Empty DataFrame
Columns: [Predicted Price]
Index: []
---------------
0    147.20
1    125.90
2    111.85
3    153.75
4    145.55
Name: C_mkt, dtype: float64


(nan, nan)